In [1]:
import librosa
import keras
import os
import numpy as np
import scipy.io.wavfile as wav
import python_speech_features
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
conf = []
mtrx = np.arange(9).reshape(3, 3) - np.arange(9).reshape(3, 3)
posisi = ["nyala awal", "mati awal","nyala tengah","mati tengah","nyala akhir","mati akhir","multi"]

In [80]:
file = 0
no_path = 42 #28#36#50#64#94#101  #43#57#71#87
num = 2
matrix =  np.arange(9).reshape(3, 3) - np.arange(9).reshape(3, 3)

In [141]:
path = "cut/"+str(no_path)+"_uji 1.wav"
koef_mfcc = 13
audio, sfreq = librosa.load(path, sr=16000)
audio_duration =  librosa.get_duration(audio, sr=sfreq)
print(audio_duration)

no_path = no_path+1

11.968


In [142]:
def extract_data_test(start_duration, end_duration,i):
    path_new = "potongan/yafi"+str(i)+".wav"
    song = AudioSegment.from_file(path)
    song = song.set_frame_rate(16000)
    extract = song[start_duration*1000:end_duration*1000]
    extract.export(path_new, format="wav")
    #load wav
    sfreq, audio = wav.read(path_new)
    #normalisasi amplitudo
    #audio_x = 2.*(audio - np.min(audio))/np.ptp(audio)-1
    mfcc = python_speech_features.base.mfcc(audio,samplerate=16000,winlen=0.02,winstep=0.01,numcep=koef_mfcc,nfft=512,preemph=0.95,winfunc=np.hamming)
    #print(mfcc.shape)
    return mfcc

In [143]:
def create_model_1DCONV():
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=20, kernel_size=4, strides=1, padding="same", input_shape=[None, koef_mfcc]),
            keras.layers.Conv1D(filters=15, kernel_size=4, strides=1, padding="same"),
            keras.layers.Conv1D(filters=15, kernel_size=4, strides=1, padding="same"),
            keras.layers.GlobalMaxPooling1D(),
            keras.layers.Dense(3, activation='softmax')
            ])
    return model

In [144]:
model = create_model_1DCONV()
model.load_weights("conv1d_model_koef_"+str(koef_mfcc)+".h5")

In [145]:
if file > 6:
    file = 0
else: file = file
print(file)
y_true=[]
if posisi[file] == "nyala awal":
    y_true = [0,2,2,2,2]
elif posisi[file] == "nyala tengah":
    y_true = [2,0,2,2,2,2]
elif posisi[file] ==  "nyala akhir":
    y_true = [2,2,2,2,2,0]
elif posisi[file] == "mati awal":
    y_true = [1,2,2,2,2]
elif posisi[file] == "mati tengah":
    y_true = [2,1,2,2,2,2]
elif posisi[file] ==  "mati akhir":
    y_true = [2,2,2,2,2,1]
else:
    y_true = [2,0,2,2,2,2,2,1,2,2,2,2]
y_pred=[]
hasil=[]
file = file+1

6


In [146]:
kelas = ["nyala", "mati", "lain"]
for i in range(1, math.ceil(audio_duration)+1):
    end_duration = i
    start_duration = i-1
    
    end_duration += 0.15
    start_duration-=0.01
    
    if start_duration<0 : start_duration = 0
    
    if end_duration>audio_duration : end_duration = audio_duration
    

    if end_duration>audio_duration : end_duration=audio_duration
    
    audio = extract_data_test(start_duration, end_duration)
    shape = audio.shape
    audio_p = audio.reshape(1, shape[0], shape[1])
    hasil = model.predict(audio_p)
    inhasil = np.argmax(hasil)
    y_pred.append(inhasil)
    print("detik :"+str(start_duration)+"-"+str(end_duration)+" ="+ kelas[inhasil])
        

detik :0-1.15 =lain
detik :0.99-2.15 =nyala
detik :1.99-3.15 =lain
detik :2.99-4.15 =lain
detik :3.99-5.15 =nyala
detik :4.99-6.15 =nyala
detik :5.99-7.15 =lain
detik :6.99-8.15 =mati
detik :7.99-9.15 =lain
detik :8.99-10.15 =lain
detik :9.99-11.15 =lain
detik :10.99-11.968 =nyala


In [147]:
print(y_true)
print(y_pred)

[2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2]
[2, 0, 2, 2, 0, 0, 2, 1, 2, 2, 2, 0]


In [148]:
confusion_matrix(y_true, y_pred, labels =[0,1,2])

array([[1, 0, 0],
       [0, 1, 0],
       [3, 0, 7]], dtype=int64)

In [149]:
matrix = matrix + confusion_matrix(y_true, y_pred, labels =[0,1,2])
print(matrix)

[[ 4  0  0]
 [ 0  3  1]
 [12  1 25]]


In [150]:
from numpy import asarray
from numpy import savetxt
savetxt("cnn 13 "+str(num)+'.csv',matrix,delimiter = '/')

In [151]:
def counts_from_confusion(confusion):
    """
    Obtain TP, FN FP, and TN for each class in the confusion matrix
    """

    counts_list = []

    # Iterate through classes and store the counts
    for i in range(confusion.shape[0]):
        tp = confusion[i, i]

        fn_mask = np.zeros(confusion.shape)
        fn_mask[i, :] = 1
        fn_mask[i, i] = 0
        fn = np.sum(np.multiply(confusion, fn_mask))

        fp_mask = np.zeros(confusion.shape)
        fp_mask[:, i] = 1
        fp_mask[i, i] = 0
        fp = np.sum(np.multiply(confusion, fp_mask))

        tn_mask = 1 - (fn_mask + fp_mask)
        tn_mask[i, i] = 0
        tn = np.sum(np.multiply(confusion, tn_mask))

        counts_list.append({'Class': i,
                            'TP': tp,
                            'FN': fn,
                            'FP': fp,
                            'TN': tn})

    return counts_list

In [152]:
counts_from_confusion(matrix)

[{'Class': 0, 'TP': 4, 'FN': 0.0, 'FP': 12.0, 'TN': 30.0},
 {'Class': 1, 'TP': 3, 'FN': 1.0, 'FP': 1.0, 'TN': 41.0},
 {'Class': 2, 'TP': 25, 'FN': 13.0, 'FP': 1.0, 'TN': 7.0}]

In [153]:
savetxt("cnn 13 conf"+str(num)+'.csv',counts_from_confusion(matrix),delimiter = '/', fmt='%s')

In [154]:
mtrx = np.append(mtrx,matrix)
mtrx.reshape(int(len(mtrx)/3),3)

array([[ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 4,  0,  0],
       [ 0,  4,  0],
       [ 3,  0, 35],
       [ 4,  0,  0],
       [ 0,  3,  1],
       [12,  1, 25]], dtype=int64)

In [155]:
mtrx.reshape(int(len(mtrx)/3),3).shape

(9, 3)

In [156]:
savetxt("cnn 13.csv",mtrx.reshape(int(len(mtrx)/3),3),delimiter = '/')